# Análisis de Datos Espaciales (ADE13): Estadísticos de Asociación Espacial Local (Getis-Ord $G_i^*$)

## En esta notebook...

Habiendo explorado estadísticos globales de autocorrelación espacial (como el índice I de Moran) que resumen el patrón espacial de todo el mapa en un único número, ahora daremos el siguiente paso fundamental: los **estadísticos locales**. Estos nos permiten no solo detectar la presencia de autocorrelación espacial, sino identificar la **ubicación específica** de los clusters en el territorio.

Introduciremos la familia de estadísticos $G$ de Getis y Ord<sup>[3]</sup>, la herramienta fundamental para lo que se conoce como **"Hotspot Analysis"** o análisis de puntos calientes. A diferencia de los estadísticos de similitud (como LISA), los estadísticos $G$ están diseñados específicamente para identificar concentraciones espaciales de valores **altos** (hotspots) y valores **bajos** (coldspots), distinguiendo entre ambos.

---

## Índice

1. El Problema: Estadísticos Globales vs. Locales
2. Teoría: La Familia de Estadísticos $G$
3. El Estadístico $G_i(d)$ (Excluyente)
4. El Estadístico $G_i^*(d)$ (Incluyente)
5. Inferencia e Interpretación: Puntuación Z
6. Comparativa Clave: $G_i^*$ vs. Moran's I Local
7. Taller Práctico: Análisis de Hotspots en Python
8. Paso 1: Carga y Preparación de Datos (Viviendas San Diego)
9. Paso 2: Creación de Pesos Espaciales por Distancia
10. Paso 3: Cálculo de $G_i^*$ con `esda`
11. Paso 4: Visualización e Interpretación (Hotspots y Coldspots)
12. Conclusiones

---

## Importación de Bibliotecas

Además de nuestras bibliotecas habituales (`geopandas`, `matplotlib`), hoy sumaremos dos componentes clave de la familia **PySAL** (Python Spatial Analysis Library):

- **`libpysal.weights`**: Para la creación de matrices de pesos espaciales basadas en criterios de proximidad (distancia, contigüidad, etc.).
- **`esda`** (Exploratory Spatial Data Analysis): Contiene las implementaciones de los estadísticos de autocorrelación espacial, tanto globales como locales, incluyendo los estadísticos $G_i$ y $G_i^*$ de Getis-Ord.

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import libpysal.weights as weights
import esda

# Configuración de visualización
plt.rcParams['figure.dpi'] = 100
pd.set_option('display.precision', 3)

## 1. El Problema: Estadísticos Globales vs. Locales

### La Limitación de los Estadísticos Globales

Los estadísticos globales de autocorrelación espacial, como el índice I de Moran global o el estadístico $G(d)$ global, condensan toda la información espacial de un mapa en un **único número**. Este valor nos indica si existe autocorrelación espacial en el conjunto de datos, pero presenta una limitación crítica: **no puede identificar *dónde* están ubicados los clusters**.

Por ejemplo, un valor de I de Moran global positivo nos dice que hay autocorrelación positiva (valores similares están agrupados), pero no nos dice:
- ¿Cuáles son las regiones que forman clusters?
- ¿Son clusters de valores altos o de valores bajos?
- ¿Hay múltiples clusters o solo uno?

### El Peligro de Ignorar la Autocorrelación

El fracaso en tener en cuenta la autocorrelación espacial puede llevar a serios errores de interpretación en análisis estadísticos<sup>[11]</sup>. Sin estadísticos locales, podemos pasar por alto patrones espaciales críticos que tienen implicaciones teóricas y prácticas importantes.

### La Solución: Estadísticos Locales

El objetivo de los estadísticos $G$ es, precisamente, detectar "bolsones" (*pockets*) locales de dependencia espacial que pueden no ser evidentes al usar un estadístico global<sup>[8]</sup>. Estos estadísticos nos permiten responder la pregunta: **"¿Dónde están los clusters y qué características tienen?"**

## 2. Teoría: La Familia de Estadísticos $G$

### Introducción

La familia de estadísticos $G$ de Getis y Ord constituye una medida de asociación espacial diseñada específicamente para analizar la concentración de valores de una variable $X$ en el espacio<sup>[3]</sup>.

### Requisito Fundamental de los Datos

Un requisito clave para aplicar estos estadísticos es que **la variable debe tener un origen natural y ser positiva**<sup>[40, 97, 331]</sup>. Esto incluye variables como:
- Precios de viviendas
- Tasas de enfermedad o mortalidad
- Conteos de eventos (crímenes, accidentes)
- Ingresos o salarios
- Densidades poblacionales

Variables que pueden tomar valores negativos o que son diferencias (sin origen natural) no son apropiadas para estos estadísticos.

### Matriz de Pesos Espaciales

El estadístico $G$ se basa en una **matriz de pesos espaciales** $w_{ij}(d)$ que define qué ubicaciones $j$ están "cerca" de la ubicación $i$<sup>[46]</sup>. El parámetro $d$ representa una **distancia umbral**: dos ubicaciones se consideran vecinas si están dentro de una distancia $d$.

Esta matriz define la **estructura de vecindad** que usaremos para evaluar si los valores de $X$ en ubicaciones cercanas están concentrados (son todos altos o todos bajos) o dispersos.

## 3. El Estadístico $G_i(d)$ (Excluyente)

### Definición

El estadístico $G_i(d)$ se define como<sup>[43]</sup>:

$$G_{i}(d)=\frac{\sum_{j=1}^{n}w_{ij}(d)x_{j}}{\sum_{j=1}^{n}x_{j}}, \quad j \neq i$$

### Componentes de la Fórmula

Vamos a descomponer cada elemento de esta expresión:

- **$w_{ij}(d)$**: Es una matriz de pesos espaciales binaria (simétrica), donde $w_{ij}=1$ si la ubicación $j$ está dentro de una distancia $d$ de la ubicación $i$, y $w_{ij}=0$ si no lo está<sup>[46]</sup>.

- **$x_j$**: Es el valor de la variable de interés en la ubicación $j$.

- **$j \neq i$**: Esta condición es **crítica**. Esta versión del estadístico **excluye** el valor del propio punto $i$ (es decir, $x_i$) tanto del numerador como del denominador<sup>[44, 48, 49]</sup>. Por eso se le llama la versión "excluyente".

- **Numerador**: Suma los valores de $X$ en los vecinos de $i$ (dentro de la distancia $d$).

- **Denominador**: Suma de todos los valores de $X$ en el conjunto de datos (excluyendo $x_i$).

### Interpretación Intuitiva

El estadístico $G_i$ compara la proporción de la suma total de $X$ que se concentra en el vecindario de $i$. Si esta proporción es significativamente mayor de lo esperado por azar, indica un **hotspot** (concentración de valores altos).

### Hipótesis Nula

La hipótesis nula ($H_0$) es la **independencia espacial**. Bajo $H_0$, cualquier permutación de los valores $x_j$ (excepto $x_i$, que se mantiene fijo en su ubicación) es igualmente probable<sup>[51]</sup>. Esto significa que no hay razón para esperar que los valores en el vecindario de $i$ sean sistemáticamente más altos o más bajos que en cualquier otra parte.

### Distribución Teórica: Expectativa y Varianza

Para realizar inferencia estadística, necesitamos conocer la distribución de $G_i$ bajo la hipótesis nula. Getis y Ord derivan las fórmulas para la expectativa y la varianza<sup>[64, 75]</sup>:

**Expectativa:**

$$E(G_{i}) = \frac{W_i}{(n-1)}$$

**Varianza:**

$$\text{Var}(G_{i})=\frac{W_{i}(n-1-W_{i})}{(n-1)^{2}(n-2)}\left(\frac{Y_{i2}}{Y_{i1}^{2}}\right)$$

Donde<sup>[65, 75]</sup>:

- $W_i = \sum_{j} w_{ij}$ (suma de los pesos para la ubicación $i$, es decir, el número de vecinos)
- $Y_{i1} = \sum_{j \neq i} x_j$ (suma de todos los valores excepto $x_i$)
- $Y_{i2} = \sum_{j \neq i} x_j^2$ (suma de los cuadrados de todos los valores excepto $x_i$)

## 4. El Estadístico $G_i^*(d)$ (Incluyente)

### Introducción de la Variante $G_i^*$

Existe una variación del estadístico $G_i$ que es **más utilizada en la práctica**: el estadístico $G_i^*(d)$<sup>[89, 92]</sup>. Esta es la versión que encontraremos implementada en la mayoría de software GIS y en librerías como `esda`.

### La Diferencia Clave

La diferencia fundamental entre $G_i$ y $G_i^*$ es que esta última versión **incluye** el valor del propio punto $i$ en el análisis<sup>[92, 93]</sup>. Es decir, en la fórmula, ahora $j$ **puede** ser igual a $i$. Por eso se le llama la versión "incluyente" o "con estrella" (*star*).

### Fórmulas para $G_i^*(d)$

Las fórmulas correspondientes a $G_i^*$ (Tabla 1 del paper) son:

**Estadístico:**

$$G_{i}^{*}(d)=\frac{\sum_{j}w_{ij}(d)x_{j}}{\sum_{j}x_{j}}$$

Notar que ahora la sumatoria del numerador incluye todos los $j$ (incluyendo $i$), y el denominador es la suma de **todos** los valores de $X$.

**Expectativa:**

$$E(G_{i}^{*}) = \frac{W_i^*}{n}$$

**Varianza:**

$$\text{Var}(G_{i}^{*})=\frac{W_{i}^{*}(n-W_{i}^{*})Y_{i2}^{*}}{n^{2}(n-1)(Y_{i1}^{*})^{2}}$$

Donde:
- $W_i^* = \sum_{j} w_{ij} + 1$ (suma de los pesos incluyendo al propio $i$)
- $Y_{i1}^* = \sum_{j} x_j$ (suma de **todos** los valores, incluyendo $x_i$)
- $Y_{i2}^* = \sum_{j} x_j^2$ (suma de los cuadrados de **todos** los valores)

### ¿Por qué usar $G_i^*$?

La versión $G_i^*$ es generalmente preferida porque:

1. **Incluye toda la información**: El valor del punto focal $i$ es parte del análisis, lo cual tiene sentido si queremos identificar si ese punto pertenece a un hotspot o coldspot.

2. **Más poder estadístico**: Al incluir más información (el valor de $x_i$), el estadístico puede detectar patrones con mayor sensibilidad.

3. **Interpretación más intuitiva**: Preguntamos "¿el punto $i$ y sus vecinos forman un cluster de valores altos/bajos?" en lugar de "¿los vecinos de $i$ (sin incluir a $i$) forman un cluster?".

En el resto de este notebook, trabajaremos con $G_i^*$, que es la versión estándar para "Hotspot Analysis".

## 5. Inferencia e Interpretación: Puntuación Z

### Del Estadístico Crudo al Z-score

En la práctica, no interpretamos directamente el valor $G_i$ o $G_i^*$ crudo. En su lugar, asumiendo normalidad de la distribución bajo la hipótesis nula<sup>[107]</sup>, calculamos el **Z-score** (valor estandarizado) del estadístico<sup>[108]</sup>:

$$Z_{i}=\frac{G_{i}(d)-E[G_{i}(d)]}{\sqrt{\text{Var}(G_{i}(d))}}$$

Esta misma fórmula aplica para $G_i^*$, usando su expectativa y varianza correspondientes.

### ¿Qué es el Z-score?

El Z-score nos dice cuántas **desviaciones estándar** el valor observado de $G_i$ está alejado de su valor esperado bajo independencia espacial. Un Z-score:

- **Cercano a 0**: El patrón observado es consistente con aleatoriedad espacial (no significativo).
- **Muy positivo** (ej. > 1.96): El valor observado es significativamente **mayor** que lo esperado por azar.
- **Muy negativo** (ej. < -1.96): El valor observado es significativamente **menor** que lo esperado por azar.

### Interpretación Crítica: Hotspots vs. Coldspots

La interpretación de los Z-scores de $G_i^*$ es fundamental para el análisis de hotspots:

#### **Hotspot (Cluster de valores ALTOS)**

Un Z-score significativamente **positivo** (por ejemplo, $Z_i > 1.96$ para un nivel de significancia del 5%) implica una **concentración espacial de valores altos** de $X$ alrededor del punto $i$<sup>[111]</sup>. 

Esto significa que tanto el punto $i$ como sus vecinos tienen valores de $X$ que son altos en comparación con el resto del área de estudio. En el contexto de precios de vivienda, esto identifica **zonas caras**.

#### **Coldspot (Cluster de valores BAJOS)**

Un Z-score significativamente **negativo** (por ejemplo, $Z_i < -1.96$) implica una **concentración espacial de valores bajos** (*small values*) de $X$ alrededor del punto $i$<sup>[112]</sup>.

Esto significa que tanto el punto $i$ como sus vecinos tienen valores de $X$ que son bajos en comparación con el resto del área de estudio. En precios de vivienda, identifica **zonas baratas**.

#### **No significativo**

Un Z-score cercano a 0 o que no supera los umbrales críticos (ej. $|Z_i| < 1.96$) indica que el patrón en esa ubicación es **consistente con aleatoriedad espacial**. No hay evidencia de un cluster.

### Umbrales de Significancia

Los umbrales comunes para los Z-scores son:

- **$|Z_i| > 1.96$**: Significancia al 5% (p < 0.05)
- **$|Z_i| > 2.58$**: Significancia al 1% (p < 0.01)
- **$|Z_i| > 1.65$**: Significancia al 10% (p < 0.10)

## 6. Comparativa Clave: $G_i^*$ vs. Moran's I Local (LISA)

### Una Distinción Teórica Fundamental

Existe una diferencia conceptual profunda entre el estadístico $G_i^*$ de Getis-Ord y el índice I de Moran Local (LISA), que es crucial comprender para elegir la herramienta adecuada<sup>[301]</sup>.

### Moran's I Local (LISA): Mide **Similitud**

El índice I de Moran Local evalúa si una ubicación y sus vecinos tienen valores **similares** (autocorrelación). Específicamente:

- **I positivo alto** → Los vecinos son **similares** al punto focal.
  - Esto puede ser un cluster **High-High (HH)**: punto alto rodeado de valores altos
  - O un cluster **Low-Low (LL)**: punto bajo rodeado de valores bajos
  
- **I negativo alto** → Los vecinos son **diferentes** al punto focal.
  - Outliers **High-Low (HL)**: punto alto rodeado de valores bajos
  - Outliers **Low-High (LH)**: punto bajo rodeado de valores altos

**Limitación clave**: Moran's I no puede distinguir entre un cluster HH y un cluster LL basándose solo en el signo del estadístico. Ambas situaciones producen un I positivo<sup>[322]</sup>. Para distinguirlos, debemos examinar el valor de $X$ en esa ubicación.

### Getis-Ord $G_i^*$: Mide **Concentración de Magnitud**

El estadístico $G_i^*$ está diseñado **específicamente** para distinguir entre concentraciones de valores altos y bajos:

- **HH (High-High)**: Produce un Z-score **positivo** (Hotspot)<sup>[304, 308]</sup>
- **LL (Low-Low)**: Produce un Z-score **negativo** (Coldspot)<sup>[306, 321]</sup>
- **HL (High-Low)**: También produce un Z-score **negativo** (pero el I de Moran sería fuertemente negativo)<sup>[318]</sup>

**Ventaja clave**: El signo del Z-score nos dice inmediatamente si estamos ante una concentración de valores **altos** (positivo) o **bajos** (negativo).

### Comparación Directa (Tabla 2 del Paper)

| Tipo de Patrón | Moran's I Local | Getis-Ord $G_i^*$ |
|----------------|-----------------|-------------------|
| High-High (HH) | Positivo (+)    | Positivo (+)      |
| Low-Low (LL)   | Positivo (+)    | Negativo (-)      |
| High-Low (HL)  | Negativo (-)    | Negativo (-)      |
| Low-High (LH)  | Negativo (-)    | Positivo (+)      |

### ¿Cuándo usar cada uno?

Ambos estadísticos son complementarios y deben usarse en conjunto<sup>[8, 324]</sup>:

- **Moran's I**: Para detectar autocorrelación espacial (similitud) y outliers espaciales.
- **Getis-Ord $G_i^*$**: Para identificar las **características de magnitud** de esos clusters (¿son hotspots o coldspots?).

En el contexto de análisis de hotspots (identificar zonas de concentración de valores altos vs. bajos), $G_i^*$ es generalmente más útil y directo en su interpretación.

## 7. Taller Práctico: Análisis de Hotspots en Python

### Caso de Estudio: Precios de Vivienda en San Diego

Vamos a replicar conceptualmente el **segundo ejemplo empírico** del paper de Getis y Ord: el análisis de precios medios de vivienda en el área metropolitana de San Diego, California, agregados por código postal (ZIP code)<sup>[339, 404]</sup>.

### Motivación del Análisis

En su análisis preliminar, Getis y Ord calcularon los estadísticos globales $G(d)$ e $I(d)$ para estos datos. Los resultados fueron:

- **$I(d)$ global**: Positivo (indicando autocorrelación positiva general)
- **$G(d)$ global**: Negativo (indicando predominancia de valores bajos)<sup>[415-417]</sup>

Sin embargo, estos estadísticos globales **no revelaron claramente** la estructura espacial subyacente. ¿Dónde están los clusters? ¿Hay zonas de precios altos y zonas de precios bajos?

### Objetivo

El objetivo de aplicar el estadístico local $G_i^*(d)$ es identificar los **"bolsones" (pockets)** de precios altos (hotspots) y precios bajos (coldspots) que los estadísticos globales no pueden mostrar<sup>[415-417]</sup>.

### Datos

Trabajaremos con los datos del **Apéndice** del paper<sup>[473]</sup>, que incluye:
- **Códigos postales** de San Diego
- **Coordenadas** (X, Y) en millas desde un origen arbitrario
- **Precio medio de vivienda** en miles de USD (1990)

¡Comencemos con la implementación!

## 8. Paso 1: Carga y Preparación de Datos (Viviendas San Diego)

Para que el notebook sea autocontenido, cargaremos una muestra de los datos presentados en el **Apéndice (página 17)** del paper<sup>[473]</sup>. Los datos incluyen las coordenadas en millas y el precio medio de vivienda en miles de USD para 24 códigos postales del área metropolitana de San Diego.

In [ ]:
# Datos extraídos del Apéndice (p. 17) del paper [473]
data = {
    'zip_code': ['92024', '92007', '92075', '92014', '92127', '92129', '92128', '92064',
                 '92037', '92122', '92117', '92109', '92110', '92111', '92123', '92108',
                 '92103', '92104', '92105', '92113', '92102', '92107', '92106', '92118'],
    'neighborhood': ['Encinitas', 'Cardiff', 'Solana Beach', 'Del Mar', 'Lake Hodges', 'R. Penasquitos', 'R. Bernardo', 'Poway',
                     'La Jolla', 'University City', 'Clairemont', 'Beaches', 'Bay Park', 'Kearny Mesa', 'Mission Village', 'Mission Valley',
                     'Hillcrest', 'North Park', 'East San Diego', 'Logan Heights', 'East San Diego', 'Ocean Beach', 'Point Loma', 'Coronado'],
    'x': [1, 2, 3, 5, 10, 12, 15, 17, 3, 6, 6, 4, 6, 8, 10, 9, 8, 11, 13, 11, 12, 3, 3, 7],
    'y': [39, 36, 34, 32, 34, 32, 35, 32, 22, 23, 20, 18, 15, 19, 19, 16, 14, 14, 14, 10, 12, 14, 12, 10],
    'price': [264, 260, 261, 309, 265, 194, 191, 236, 398, 201, 192, 249, 152, 138, 131, 89, 225, 152, 111, 84, 88, 229, 338, 374]
}
df = pd.DataFrame(data)

# Convertir a GeoDataFrame usando las coordenadas en millas
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.x, df.y)
)

# Visualizar los primeros registros
print("Primeros registros del dataset:")
print(gdf[['neighborhood', 'x', 'y', 'price']].head(10))
print(f"\nTotal de códigos postales: {len(gdf)}")
print(f"Precio promedio: ${gdf['price'].mean():.1f}k USD")
print(f"Rango de precios: ${gdf['price'].min():.0f}k - ${gdf['price'].max():.0f}k USD")

In [ ]:
# Visualizar los datos base (similar a la Figura 3 del paper [430])
fig, ax = plt.subplots(figsize=(10, 12))
gdf.plot(column='price', ax=ax, legend=True, cmap='viridis', s=100,
         legend_kwds={'label': "Precio de Vivienda (Miles USD)", 'orientation': "horizontal"})

# Anotar algunos puntos clave para referencia
for x, y, label, price in zip(gdf.geometry.x, gdf.geometry.y, gdf.neighborhood, gdf.price):
    if label in ['La Jolla', 'Point Loma', 'Mission Valley', 'East San Diego', 'Coronado', 'Del Mar']:
        ax.annotate(f'{label}\n${price}k', (x, y), textcoords="offset points", 
                    xytext=(3, 3), ha='left', fontsize=8, 
                    bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7))

ax.set_title('Precios de Vivienda en San Diego por Código Postal [473]', fontsize=14, fontweight='bold')
plt.xlabel('Coordenada X (millas)', fontsize=11)
plt.ylabel('Coordenada Y (millas)', fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 9. Paso 2: Creación de Pesos Espaciales por Distancia

### Matriz de Pesos Basada en Distancia

Los estadísticos $G_i$ requieren una matriz de pesos espaciales $w_{ij}(d)$ basada en distancia<sup>[46]</sup>. En el paper, el análisis de $G_i^*$ para San Diego se realizó con un **umbral de distancia** $d=5$ **millas**<sup>[418, 460]</sup>.

Vamos a replicar esto usando `libpysal.weights.DistanceBand`, que crea una matriz binaria donde:

$$w_{ij} = \begin{cases} 1 & \text{si } \text{distancia}(i,j) \leq d \\ 0 & \text{si } \text{distancia}(i,j) > d \end{cases}$$

Esta matriz define qué códigos postales se consideran "vecinos" de cada código postal focal.

In [ ]:
# Definir el umbral de distancia (d) en 5 millas, como en el paper [460]
d_threshold = 5.0

# Crear la matriz de pesos basada en banda de distancia
# 'binary=True' es el default y replica la matriz {0,1} del paper
W = weights.DistanceBand.from_dataframe(gdf, d_threshold, binary=True)

# Es importante que W no esté estandarizada por fila para este estadístico
W.transform = 'B'  # 'B' for binary (sin estandarizar)

print(f"Matriz de pesos creada con umbral d = {d_threshold} millas")
print(f"Número de ubicaciones: {W.n}")
print(f"Promedio de vecinos por ubicación: {W.mean_neighbors:.2f}")
print(f"Rango de vecinos: {W.min_neighbors} - {W.max_neighbors}")

# Inspeccionemos los vecinos de 'Point Loma' (ZIP 92106)
point_loma_idx = gdf[gdf['neighborhood'] == 'Point Loma'].index[0]
neighbors_idx = W.neighbors[point_loma_idx]

print(f"\nVecinos de 'Point Loma' (a {d_threshold} millas o menos):")
if len(neighbors_idx) > 0:
    print(gdf.loc[neighbors_idx][['neighborhood', 'price']])
else:
    print("No tiene vecinos dentro del umbral.")

## 10. Paso 3: Cálculo de $G_i^*$ con `esda`

Ahora aplicaremos el estadístico $G_i^*$ usando la función `esda.G_Local`. 

**Parámetros clave:**
- `y`: La variable de interés (precios de vivienda)
- `W`: La matriz de pesos espaciales
- `star=True`: **Fundamental** para usar la versión $G_i^*$ (incluyente) como en la Tabla 1 y Figura 4 del paper<sup>[92, 460]</sup>

La función calculará automáticamente los Z-scores y p-values para cada ubicación.

In [ ]:
# Variable a analizar (debe ser un array de numpy)
y = gdf['price'].values

# Calcular el estadístico G_Local (G_i*)
# star=True indica que usamos la versión G_i* (incluye el valor propio)
g_local_star = esda.G_Local(y, W, star=True)

# Añadir los Z-scores (Zs) y p-values (p_sim) al GeoDataFrame
gdf['G_star'] = g_local_star.Gs  # Valores crudos del estadístico
gdf['G_star_Zs'] = g_local_star.Zs  # Z-scores
gdf['G_star_p_sim'] = g_local_star.p_sim  # p-values basados en simulación (permutaciones)

# Ver los resultados, ordenados por Z-score (de mayor a menor)
print("=" * 80)
print("RESULTADOS DEL ANÁLISIS G_i* (d=5 millas)")
print("=" * 80)
result_df = gdf[['neighborhood', 'price', 'G_star_Zs', 'G_star_p_sim']].copy()
result_df = result_df.sort_values('G_star_Zs', ascending=False)
result_df['significance'] = result_df['G_star_p_sim'].apply(
    lambda p: '***' if p < 0.01 else ('**' if p < 0.05 else ('*' if p < 0.10 else 'ns'))
)
print(result_df.to_string(index=False))
print("\nSignificancia: *** p<0.01, ** p<0.05, * p<0.10, ns = no significativo")

## 11. Paso 4: Visualización e Interpretación (Hotspots y Coldspots)

### Estrategia de Visualización

Vamos a visualizar nuestros resultados replicando la lógica de la **Figura 4** del paper<sup>[460]</sup>. Crearemos dos mapas complementarios:

1. **Mapa de Z-scores crudos**: Muestra la distribución espacial continua de los valores. Los colores rojos indican Z-scores positivos (tendencia a hotspot) y los azules indican Z-scores negativos (tendencia a coldspot).

2. **Mapa de significancia estadística**: Filtra solo los códigos postales donde el Z-score es extremo ($|Z_i| > 1.96$) **y** el p-value es estadísticamente significativo ($p < 0.05$). Este es el mapa que identifica definitivamente los hotspots y coldspots.

In [ ]:
# Mapa 1: Visualización de los Z-scores
fig, ax = plt.subplots(figsize=(10, 12))

# Usamos un mapa de color divergente (Rojo-Azul)
# 'coolwarm' o 'RdBu_r' son buenas opciones
# Rojo = valores positivos (hotspots), Azul = valores negativos (coldspots)
gdf.plot(
    column='G_star_Zs',
    cmap='coolwarm',
    legend=True,
    ax=ax,
    s=120,
    edgecolor='black',
    linewidth=0.5,
    legend_kwds={'label': "Z-score $G_i^*$ (d=5 millas)", 'shrink': 0.8}
)

# Anotar los valores Z en el mapa
for x, y, label in zip(gdf.geometry.x, gdf.geometry.y, gdf.G_star_Zs):
    ax.annotate(f"{label:.2f}", (x, y), textcoords="offset points", 
                xytext=(0, -8), ha='center', fontsize=7, 
                bbox=dict(boxstyle='round,pad=0.2', facecolor='white', alpha=0.6))

ax.set_title('Análisis $G_i^*$ de Precios de Vivienda: Z-scores\n(Rojo = Hotspots, Azul = Coldspots)', 
             fontsize=13, fontweight='bold')
plt.xlabel('Coordenada X (millas)', fontsize=11)
plt.ylabel('Coordenada Y (millas)', fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Interpretación del Mapa de Z-scores

En el mapa anterior observamos:

- **Z-scores fuertemente positivos (rojo oscuro)**: Se concentran en las zonas **costeras** del área metropolitana, particularmente en códigos postales como **La Jolla**, **Point Loma**, **Coronado** y **Del Mar**. Estos son indicadores de **hotspots** de precios altos.

- **Z-scores fuertemente negativos (azul oscuro)**: Se observan en zonas más **centrales/orientales** como **Mission Valley**, **East San Diego**, **Logan Heights** y **Kearny Mesa**. Estos son indicadores de **coldspots** de precios bajos.

- **Z-scores cercanos a cero (blanco/amarillo)**: Representan áreas donde el patrón espacial es consistente con aleatoriedad.

Esto coincide exactamente con los hallazgos del paper, que identificó los distritos costeros como positivamente asociados (hotspots) y los distritos centrales/interiores como negativamente asociados (coldspots)<sup>[434, 436, 447]</sup>.

In [ ]:
# Mapa 2: Visualización de Significancia Estadística
# Nivel de significancia
alpha = 0.05
z_critical = 1.96

# Crear una columna para la clasificación del cluster
gdf['cluster_type'] = 'No Significativo'
gdf.loc[(gdf['G_star_Zs'] > z_critical) & (gdf['G_star_p_sim'] < alpha), 'cluster_type'] = 'Hotspot (HH)'
gdf.loc[(gdf['G_star_Zs'] < -z_critical) & (gdf['G_star_p_sim'] < alpha), 'cluster_type'] = 'Coldspot (LL)'

# Contar clusters
n_hotspots = (gdf['cluster_type'] == 'Hotspot (HH)').sum()
n_coldspots = (gdf['cluster_type'] == 'Coldspot (LL)').sum()
n_ns = (gdf['cluster_type'] == 'No Significativo').sum()

print(f"Clusters identificados (α=0.05, |Z| > 1.96):")
print(f"  - Hotspots (HH): {n_hotspots}")
print(f"  - Coldspots (LL): {n_coldspots}")
print(f"  - No significativos: {n_ns}")

# Mapear los clusters significativos
fig, ax = plt.subplots(figsize=(10, 12))

# Colores para el mapa categórico
colors = {'Hotspot (HH)': 'red', 'Coldspot (LL)': 'blue', 'No Significativo': 'lightgrey'}
gdf['color'] = gdf['cluster_type'].map(colors)

gdf.plot(
    color=gdf['color'],
    ax=ax,
    s=150,
    edgecolor='black',
    linewidth=1.0
)

# Anotar los barrios significativos
for x, y, label, cluster in zip(gdf.geometry.x, gdf.geometry.y, gdf.neighborhood, gdf.cluster_type):
    if cluster != 'No Significativo':
        ax.annotate(label, (x, y), textcoords="offset points", 
                    xytext=(3, 3), ha='left', fontsize=9,
                    bbox=dict(boxstyle='round,pad=0.3', 
                              facecolor='white', 
                              edgecolor='black',
                              alpha=0.8))

# Leyenda manual
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='red', edgecolor='black', label=f'Hotspot (HH) - n={n_hotspots}'),
    Patch(facecolor='blue', edgecolor='black', label=f'Coldspot (LL) - n={n_coldspots}'),
    Patch(facecolor='lightgrey', edgecolor='black', label=f'No Significativo - n={n_ns}')
]
ax.legend(handles=legend_elements, loc='upper left', fontsize=10, title='Tipo de Cluster (p<0.05)')

ax.set_title('Hotspots y Coldspots Estadísticamente Significativos\nPrecios de Vivienda en San Diego', 
             fontsize=13, fontweight='bold')
plt.xlabel('Coordenada X (millas)', fontsize=11)
plt.ylabel('Coordenada Y (millas)', fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Interpretación del Mapa de Significancia

El mapa de significancia confirma la estructura de **"bolsones" (pockets)** que el paper describe:

#### Hotspots Identificados (Rojo)
Los códigos postales que forman hotspots estadísticamente significativos están concentrados en la **zona costera**:
- **La Jolla** (92037): La zona más cara ($398k), formando un hotspot muy fuerte
- **Point Loma** (92106): Zona costera premium ($338k)
- **Coronado** (92118): Isla exclusiva ($374k)
- **Del Mar** (92014): Comunidad costera al norte ($309k)

Estos son los "bolsones de precios altos" donde tanto el punto focal como sus vecinos tienen valores significativamente superiores al promedio regional.

#### Coldspots Identificados (Azul)
Los códigos postales que forman coldspots están en zonas más **interiores/centrales**:
- **Mission Valley** (92108): Centro geográfico con precios bajos ($89k)
- **East San Diego** (92105, 92102): Zona este con precios deprimidos ($111k, $88k)
- **Logan Heights** (92113): Zona sur-central ($84k)
- **Kearny Mesa** (92111): Zona industrial/comercial ($138k)

Estos son los "bolsones de precios bajos" donde el punto focal y sus vecinos tienen valores significativamente inferiores al promedio.

#### El Poder de los Estadísticos Locales

Este es el poder de los estadísticos $G_i^*$: **revelan patrones locales que los estadísticos globales no pueden mostrar**. Recordemos que:

- El **$G(d)$ global** fue **negativo**<sup>[416]</sup>, indicando predominancia general de valores bajos
- El **$I(d)$ global** fue **positivo**<sup>[415]</sup>, indicando autocorrelación espacial general

Pero ninguno de estos dos números nos dice *dónde* están los clusters ni distingue entre hotspots y coldspots. Los estadísticos locales $G_i^*$ nos dicen **por qué** el $G$ global fue negativo: porque el cluster de valores bajos (coldspots en el centro) era espacialmente más influyente que el cluster de valores altos (hotspots costeros)<sup>[416, 436]</sup>.

## 12. Conclusiones

### Síntesis de la Lección

En esta notebook hemos completado la transición de estadísticos **globales** a estadísticos **locales** de autocorrelación espacial, centrándonos en la familia de estadísticos $G$ de Getis y Ord.

### Conceptos Clave Aprendidos

1. **Los estadísticos $G$ nos proveen una forma directa de evaluar la asociación espacial**, tanto a nivel global como local<sup>[452]</sup>.

2. **Diferencia fundamental entre $G_i^*$ y Moran's I**: 
   - **Moran's I Local (LISA)**: Mide **similitud** (autocorrelación). No distingue entre clusters High-High y Low-Low basándose solo en el signo del estadístico.
   - **Getis-Ord $G_i^*$**: Mide **concentración de magnitud**. El signo del Z-score indica directamente si es un hotspot (positivo) o coldspot (negativo)<sup>[325, 455]</sup>.

3. **Interpretación de Z-scores**:
   - $Z_i > 1.96$ (p < 0.05): **Hotspot** (concentración de valores altos)
   - $Z_i < -1.96$ (p < 0.05): **Coldspot** (concentración de valores bajos)
   - $|Z_i| < 1.96$: No significativo (patrón aleatorio)

4. **Requisitos de los datos**: La variable debe tener un origen natural y ser positiva (precios, tasas, conteos).

### Implementación Práctica

La implementación con `libpysal` y `esda` nos permitió:
- Crear matrices de pesos espaciales basadas en distancia (`DistanceBand`)
- Calcular el estadístico $G_i^*$ con `esda.G_Local(star=True)`
- Replicar el análisis empírico del paper, identificando hotspots y coldspots de precios de vivienda en San Diego

### Lección Final

El uso de estadísticos locales ($G_i^*$) es **fundamental** en el análisis espacial. Como vimos en el ejemplo de San Diego, una medida global puede ser engañosa y ocultar patrones locales subyacentes de gran importancia<sup>[401]</sup>. 

Los estadísticos globales nos dicen **si hay** un patrón. Los estadísticos locales nos dicen **dónde está** y **qué características tiene**. Ambos son complementarios y esenciales para un análisis espacial completo.

---

### Referencias

Este notebook está basado en:

**Getis, A., & Ord, J. K. (1992).** *The Analysis of Spatial Association by Use of Distance Statistics.* Geographical Analysis, 24(3), 189-206.

Las referencias numéricas (ej. [3], [46], [460]) corresponden a páginas o elementos específicos del paper original.